In [1]:
class Character:

    def __init__(self, name: str, hp: int = 100, level: int = 1):
        """Create a character with a name, hit points (hp), and level"""
        if hp <= 0:
            raise ValueError("HP must be positive!")
        if level < 1:
            raise ValueError("Level must be at least 1!")

        self.name: str = name
        self.hp: int = hp
        self.level: int = level
        self.inventory: list = []

    @property
    def is_alive(self) -> bool:
        return self.hp > 0
    
    def __repr__(self) -> str:
        """For developers: unambiguous and ideally eval()-able"""
        return f"Character(name={self.name!r}, hp={self.hp}, level={self.level})"
    
    def __str__(self) -> str:
        """For users: readable and informative"""
        status = "💀" if not self.is_alive else "💚" if self.hp > 80 else "❤️"
        return f"{status} {self.name} (Lv.{self.level}) - {self.hp}/100 HP"

    def __eq__(self, other) -> bool:
        """Characters are equal if they have the same name and level"""
        if not isinstance(other, Character):
            return NotImplemented
        return self.name == other.name and self.level == other.level
    
    def __lt__(self, other) -> bool:
        """Compare characters by level for sorting."""
        if not isinstance(other, Character):
            return NotImplemented
        return self.level < other.level
    
    def __hash__(self):
        """Make characters hashable for use in sets/dicts"""
        return hash((self.name, self.level))


In [2]:
class Boss(Character):
    def __init__(self, name, **kwargs):
        super().__init__(name, **kwargs)
        """Initialize a boss character with default stats"""
        super().__setattr__("_stats", {"strength": 10, "agility": 10, "intelligence": 10})
        # ↑ "stats" is a common term in game development
        # and refers to a character's attributes or status values.

    def __getattr__(self, name: str) -> int:
        """Called when attribute isn't found normally"""
        stats = self.__dict__.get("_stats", {})  # Avoid triggering recursion
        if name in stats:
            return stats[name]
        raise AttributeError(f"'{self.__class__.__name__}' has no attribute '{name}'")

    def __setattr__(self, name: str, value: int):
        """Intercept stat assignments with validation."""
        stats = self.__dict__.get("_stats")
        if stats is not None and name in stats:
            if value < 0:
                raise ValueError(f"{name} cannot be negative")
            stats[name] = value
        else:
            super().__setattr__(name, value)

In [3]:
boss = Boss(name="Bowser")

In [4]:
print(boss.strength)

10


In [5]:
boss.strength = 15
print(boss.strength)

15


In [6]:
try:
    boss.agility = -5
except ValueError as error:
    print(f"Validation: {error}")

Validation: agility cannot be negative
